In [75]:
# Healthcare CSV file: diabetes.csv
# Goal: Analyze bias in healthcare data and mitigate it using DALEX python package


In [76]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import dataset
dataset = pd.read_csv('diabetes.csv')


In [77]:
print(dataset)


     Unnamed: 0  Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   
0             0          6.0      148           72.0           35.0     30.5  \
1             1          1.0       85           66.0           29.0     30.5   
2             2          8.0      183           64.0           23.0     30.5   
3             3          1.0       89           66.0           23.0     94.0   
4             4          0.0      137           40.0           35.0    168.0   
..          ...          ...      ...            ...            ...      ...   
758         763         10.0      101           76.0           23.0    180.0   
759         764          2.0      122           70.0           27.0     30.5   
760         765          5.0      121           72.0           23.0    112.0   
761         766          1.0      126           60.0           23.0     30.5   
762         767          1.0       93           70.0           31.0     30.5   

      BMI  DiabetesPedigreeFunction   A

# Models

We will make 3 predictive models


In [78]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# classifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(dataset.drop(
    'Outcome', axis=1), dataset['Outcome'], test_size=0.2, random_state=100)
categorical_features = ['AgeCategory']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_features = ['Pregnancies', 'Glucose', 'BloodPressure',
                      'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Decision Tree
dt = Pipeline(steps=[('preprocessor', preprocessor),
                     ('classifier', DecisionTreeClassifier(max_depth=7, random_state=100))])

dt.fit(X_train, y_train)

# Random Forest
rf = Pipeline(steps=[('preprocessor', preprocessor),
                     ('classifier', RandomForestClassifier(n_estimators=200, max_depth=7, random_state=100))])

rf.fit(X_train, y_train)

# Logistic Regression
lr = Pipeline(steps=[('preprocessor', preprocessor),
                     ('classifier', LogisticRegression())])

lr.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Pregnancies', 'Glucose',
                                                   'BloodPressure',
                                                   'SkinThickness', 'Insulin',
                                                   'BMI',
                                                   'DiabetesPedigreeFunction',
                                                   'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['AgeCategory'])])),
                ('classifier', LogisticRegression())])

# Explainers

dalex let us create an explainer for each model


In [79]:
import dalex as dx

exp_tree = dx.Explainer(dt, X_test, y_test, label='Decision Tree')
exp_forest = dx.Explainer(rf, X_test, y_test, label='Random Forest')
exp_logreg = dx.Explainer(lr, X_test, y_test, label='Logistic Regression')


Preparation of a new explainer is initiated

  -> data              : 153 rows 10 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 153 values
  -> model_class       : sklearn.tree._classes.DecisionTreeClassifier (default)
  -> label             : Decision Tree
  -> predict function  : <function yhat_proba_default at 0x7fd14239d480> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0, mean = 0.399, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -1.0, mean = -0.0131, max = 1.0
  -> model_info        : package sklearn

A new explainer has been created!
Preparation of a new explainer is initiated

  -> data              : 153 rows 10 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted t

# Model Performance Measures


In [80]:
pd.concat([exp.model_performance().result for exp in [
          exp_logreg, exp_tree, exp_forest]])


,recall,precision,f1,accuracy,auc
Logistic Regression,0.593220,0.700000,0.642202,0.745098,0.825279
Decision Tree,0.711864,0.677419,0.694215,0.758170,0.765056
Random Forest,0.694915,0.759259,0.725664,0.797386,0.846376


# Variable Importance

Measure the importance of each variable in the model using the variable importance function of the explainer


In [81]:
exp_tree.model_parts().plot(
    objects=[exp_forest.model_parts(), exp_logreg.model_parts()])


# Fairness: Age

We will focus on the age variable. We compute the Fairness using the model_fairness method


In [82]:
protected = np.where(X_test.Age > 50, 'old', 'young')
privileged = 'old'

mf_tree = exp_tree.model_fairness(protected=protected,
                                  privileged=privileged)
mf_forest = exp_forest.model_fairness(protected=protected,
                                      privileged=privileged)
mf_logreg = exp_logreg.model_fairness(protected=protected,
                                      privileged=privileged)


## Now Check for Bias


In [83]:
mf_tree.fairness_check()


Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'old'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
         TPR       ACC       PPV       FPR      STP
young  0.784  0.989542  0.961429  0.600601  0.64966


In [84]:
mf_forest.fairness_check()


Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'old'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
         TPR       ACC       PPV      FPR       STP
young  0.647  0.827843  0.824522  1.27027  0.625709


In [85]:
mf_logreg.fairness_check()


Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'old'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
         TPR      ACC       PPV       FPR       STP
young  0.529  0.89199  0.951857  0.423423  0.443586


### Visualize the result


In [86]:
mf_tree.plot(objects=[mf_forest, mf_logreg])


In [87]:
mf_tree.plot(objects=[mf_logreg, mf_forest], type='stacked')

The DecisionTree seems to have least parity loss

In [88]:
mf_tree.plot(objects=[mf_logreg, mf_forest],
             type="performance_and_fairness",
             fairness_metric="FPR",
             performance_metric="accuracy")

# Mitigation

In [89]:
from dalex.fairness import resample
from copy import copy

In [90]:
# copying: we consider only the decision tree classifier
clf_u = copy(dt)
clf_p = copy(dt)

## Resampling

In [91]:
# resample
indices_uniform = resample(protected, y_test, verbose=False)
indices_preferential = resample(protected,
                                y_test,
                                type='preferential',  # different type
                                probs=exp_tree.y_hat,  # requires probabilities
                                verbose=False)

clf_u.fit(X_train.iloc[indices_uniform, :], y_train.iloc[indices_uniform])
clf_p.fit(X_train.iloc[indices_preferential, :], y_train.iloc[indices_preferential])


ParameterCheckError: Parameter Check Error, protected and target (y) must have the same length

In [ ]:
# Now check if fairness is ok
exp3 = dx.Explainer(clf_u, X_test, y_test, verbose = False)
exp4 = dx.Explainer(clf_p, X_test, y_test, verbose = False)

mf_u = exp3.model_fairness(protected=protected, privileged=privileged, label="uniform")
mf_p = exp4.model_fairness(protected=protected, privileged=privileged, label="preferential")

mf_u.plot([mf_p])



In [ ]:
mf_u.fairness_check()
mf_p.fairness_check()

Bias detected in 4 metrics: TPR, ACC, PPV, FPR

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'old'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
            TPR       ACC    PPV       FPR       STP
young  1.829333  1.389414  1.272  0.705706  1.144476
Bias detected in 4 metrics: TPR, ACC, PPV, FPR

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'old'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
            TPR       ACC    PPV       FPR       STP
young  1.829333  1.389414  1.272  0.705706  1.144476
